# Análise de vendas de moveis

O dataset a ser analisado hoje contém informações sobre vendas de moveis, divididos em algumas informações importantes como categorias, sub-categoria, cidade da venda, lucro e outras.

O objetivo principal dessa análise é entender quais produtos, de quais categorias geram mais lucro e tentar entender as possiveis causas para isso.

In [1]:
import pandas as pd
import plotly.express as px
print(pd.__version__)

2.2.1


In [2]:
data = pd.read_csv("Super_Store_data.csv", encoding='latin-1')
data.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
3,6,CA-2014-115812,6/9/2014,6/14/2014,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,90032,West,FUR-FU-10001487,Furniture,Furnishings,Eldon Expressions Wood and Plastic Desk Access...,48.8600,7,0.00,14.1694
4,11,CA-2014-115812,6/9/2014,6/14/2014,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,90032,West,FUR-TA-10001539,Furniture,Tables,Chromcraft Rectangular Conference Tables,1706.1840,9,0.20,85.3092


Após uma visão geral do dataset, vemos que temos 21 colunas a serem exploradas.

Estamos lidando com dados de vendas, então com certeza é importante coletar os periodos em que cada tipo de produto foi mais vendido.

Portanto, o primeiro passo para essa análise vai ser limpar esses dados retirando quaisquer colunas que não sejam uteis para a nossa análise.

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2121 entries, 0 to 2120
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         2121 non-null   int64  
 1   Order ID       2121 non-null   object 
 2   Order Date     2121 non-null   object 
 3   Ship Date      2121 non-null   object 
 4   Ship Mode      2121 non-null   object 
 5   Customer ID    2121 non-null   object 
 6   Customer Name  2121 non-null   object 
 7   Segment        2121 non-null   object 
 8   Country        2121 non-null   object 
 9   City           2121 non-null   object 
 10  State          2121 non-null   object 
 11  Postal Code    2121 non-null   int64  
 12  Region         2121 non-null   object 
 13  Product ID     2121 non-null   object 
 14  Category       2121 non-null   object 
 15  Sub-Category   2121 non-null   object 
 16  Product Name   2121 non-null   object 
 17  Sales          2121 non-null   float64
 18  Quantity

In [4]:
data[data.duplicated()]

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit


Nossos dados já estão limpos, não possuem nenhum dado nulo e nenhum valor duplicado.
Podemos seguir para excluir as colunas que não serão utilizadas.

In [5]:
data.columns

Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State',
       'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category',
       'Product Name', 'Sales', 'Quantity', 'Discount', 'Profit'],
      dtype='object')

In [6]:
dados_limpos = data.drop(["Row ID", "Order ID", "Customer ID", "Customer Name", "Postal Code"], axis=1)
dados_limpos.head()

,Order Date,Ship Date,Ship Mode,Segment,Country,City,State,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,11/8/2016,11/11/2016,Second Class,Consumer,United States,Henderson,Kentucky,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,11/8/2016,11/11/2016,Second Class,Consumer,United States,Henderson,Kentucky,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,10/11/2015,10/18/2015,Standard Class,Consumer,United States,Fort Lauderdale,Florida,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
3,6/9/2014,6/14/2014,Standard Class,Consumer,United States,Los Angeles,California,West,FUR-FU-10001487,Furniture,Furnishings,Eldon Expressions Wood and Plastic Desk Access...,48.8600,7,0.00,14.1694
4,6/9/2014,6/14/2014,Standard Class,Consumer,United States,Los Angeles,California,West,FUR-TA-10001539,Furniture,Tables,Chromcraft Rectangular Conference Tables,1706.1840,9,0.20,85.3092


## Perguntas de negocio

O objetivo principal dessa análise é descobrir qual categoria de produto mais da lucros e os possiveis fatores que levam a isso.

In [7]:
vendas_por_categoria = dados_limpos.groupby("Category")["Sales"].mean()
vendas_por_categoria.head()

Category
Furniture    349.834887
Name: Sales, dtype: float64

Na linha de código acima fica evidente a necessidade de calma e atenção na hora da análise inicial do dataset e do tratamento dos dados. A ideia era agrupar os dados por categoria e retornar a media de vendas de cada uma, entretanto, a coluna "Category" possui apenas um valor unico e pode ser descartada.

In [8]:
dados_limpos.drop("Category", axis = 1)

,Order Date,Ship Date,Ship Mode,Segment,Country,City,State,Region,Product ID,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,11/8/2016,11/11/2016,Second Class,Consumer,United States,Henderson,Kentucky,South,FUR-BO-10001798,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,11/8/2016,11/11/2016,Second Class,Consumer,United States,Henderson,Kentucky,South,FUR-CH-10000454,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,10/11/2015,10/18/2015,Standard Class,Consumer,United States,Fort Lauderdale,Florida,South,FUR-TA-10000577,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
3,6/9/2014,6/14/2014,Standard Class,Consumer,United States,Los Angeles,California,West,FUR-FU-10001487,Furnishings,Eldon Expressions Wood and Plastic Desk Access...,48.8600,7,0.00,14.1694
4,6/9/2014,6/14/2014,Standard Class,Consumer,United States,Los Angeles,California,West,FUR-TA-10001539,Tables,Chromcraft Rectangular Conference Tables,1706.1840,9,0.20,85.3092
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2116,3/19/2015,3/22/2015,First Class,Home Office,United States,Houston,Texas,Central,FUR-BO-10004218,Bookcases,Bush Heritage Pine Collection 5-Shelf Bookcase...,383.4656,4,0.32,-67.6704
2117,12/5/2016,12/10/2016,Second Class,Corporate,United States,Newark,Delaware,East,FUR-FU-10002671,Furnishings,Electrix 20W Halogen Replacement Bulb for Zoom...,13.4000,1,0.00,6.4320
2118,9/6/2015,9/9/2015,Second Class,Consumer,United States,Lafayette,Louisiana,South,FUR-TA-10001039,Tables,KI Adjustable-Height Table,85.9800,1,0.00,22.3548
2119,1/21/2014,1/23/2014,Second Class,Consumer,United States,Miami,Florida,South,FUR-FU-10001889,Furnishings,Ultra Door Pull Handle,25.2480,3,0.20,4.1028


In [9]:
vendas_por_subcategoria = dados_limpos.groupby("Sub-Category")["Profit"].mean()
vendas_por_subcategoria

Sub-Category
Bookcases     -15.230509
Chairs         43.095894
Furnishings    13.645918
Tables        -55.565771
Name: Profit, dtype: float64

In [10]:
grafico_vendas_subcategoria = px.bar(vendas_por_subcategoria, x = vendas_por_subcategoria.index, y = vendas_por_subcategoria.values)
grafico_vendas_subcategoria.show()

Com essa primeira tabela já podemos retirar algumas informações importantes. Duas das quatro sub-categorias tiveram prejuizos nos lucros, sendo elas a de estantes e de mesas.

O produto que maior teve lucro, foi o de cadeiras.

Esse é um bom ponto de partida para nossa análise, tentar identificar o que levou esses produtos a terem prejuizos.